<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/NLP/%EB%AC%B8%EC%9E%A5%EC%9D%BC%EA%B4%80%EC%84%B1(%EC%9D%91%EC%A7%91%EC%84%B1)_%EC%BD%94%EC%82%AC%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84%EA%B8%B0%EB%B0%98_%EC%96%B8%EB%A1%A0_%EB%B8%94%EB%A1%9C%EA%B7%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sentence transfomers 구성 및 훈련

In [12]:
!pip install sentence_transformers datasets

In [13]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

In [14]:
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

In [15]:
model_name = "monologg/koelectra-base-v3-discriminator"
train_batch_size = 32
num_epochs = 4
model_save_path = "output/training_klue_sts_" + model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [16]:
embedding_model = models.Transformer(model_name)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

In [18]:
model = SentenceTransformer(modules=[embedding_model, pooler])

2021-10-28 05:58:59 - Use pytorch device: cuda


- 데이터 명 : klue-sts
- 데이터 종류 : 의미 유사도를 측정하기 위한 문장 쌍
- 데이터 구조 :
https://github.com/KLUE-benchmark/KLUE/wiki/KLUE-STS-dataset-description
- 출처 : 
https://github.com/KLUE-benchmark/KLUE

In [8]:
datasets = load_dataset("klue", "sts")

Downloading:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/sts/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
datasets["train"][0]

{'guid': 'klue-sts-v1_train_00000',
 'labels': {'binary-label': 1, 'label': 3.7, 'real-label': 3.714285714285714},
 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 'source': 'airbnb-rtt'}

In [10]:
testsets = load_dataset("kor_nlu", "sts")

Downloading:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/89.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/66.1k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset kor_nlu downloaded and prepared to /root/.cache/huggingface/datasets/kor_nlu/sts/1.0.0/4facbba77df60b0658056ced2052633e681a50187b9428bd5752ebd59d332ba8. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
train_samples = []
dev_samples = []
test_samples = []

# KLUE STS 내 훈련, 검증 데이터 예제 변환
for phase in ["train", "validation"]:
    examples = datasets[phase]

    for example in examples:
        score = float(example["labels"]["label"]) / 5.0  # 0.0 ~ 1.0 스케일로 유사도 정규화

        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]], 
            label=score,
        )

        if phase == "validation":
            dev_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

# KorSTS 내 테스트 데이터 예제 변환
for example in testsets["test"]:
    score = float(example["score"]) / 5.0

    if example["sentence1"] and example["sentence2"]:
        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]],
            label=score,
        )

    test_samples.append(inp_example)

In [12]:
train_dataloader = DataLoader(
    train_samples,
    shuffle=True,
    batch_size=train_batch_size,
)
train_loss = losses.CosineSimilarityLoss(model=model)

In [13]:
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    dev_samples,
    name="sts-dev",
)

In [14]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1)  # 10% of train data for warm-up
logging.info(f"Warmup-steps: {warmup_steps}")

2021-10-28 04:49:22 - Warmup-steps: 146


In [15]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-10-28 04:57:22 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 0:
2021-10-28 04:57:27 - Cosine-Similarity :	Pearson: 0.5927	Spearman: 0.5849
2021-10-28 04:57:27 - Manhattan-Distance:	Pearson: 0.5770	Spearman: 0.5593
2021-10-28 04:57:27 - Euclidean-Distance:	Pearson: 0.5790	Spearman: 0.5607
2021-10-28 04:57:27 - Dot-Product-Similarity:	Pearson: 0.5220	Spearman: 0.5116
2021-10-28 04:57:27 - Save model to output/training_klue_sts_monologg-koelectra-base-v3-discriminator-2021-10-28_04-48-52


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-10-28 05:05:06 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 1:
2021-10-28 05:05:12 - Cosine-Similarity :	Pearson: 0.6661	Spearman: 0.6655
2021-10-28 05:05:12 - Manhattan-Distance:	Pearson: 0.6629	Spearman: 0.6477
2021-10-28 05:05:12 - Euclidean-Distance:	Pearson: 0.6634	Spearman: 0.6486
2021-10-28 05:05:12 - Dot-Product-Similarity:	Pearson: 0.5797	Spearman: 0.5625
2021-10-28 05:05:12 - Save model to output/training_klue_sts_monologg-koelectra-base-v3-discriminator-2021-10-28_04-48-52


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-10-28 05:12:51 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 2:
2021-10-28 05:12:56 - Cosine-Similarity :	Pearson: 0.6967	Spearman: 0.6939
2021-10-28 05:12:56 - Manhattan-Distance:	Pearson: 0.6729	Spearman: 0.6589
2021-10-28 05:12:56 - Euclidean-Distance:	Pearson: 0.6741	Spearman: 0.6608
2021-10-28 05:12:56 - Dot-Product-Similarity:	Pearson: 0.6338	Spearman: 0.6189
2021-10-28 05:12:56 - Save model to output/training_klue_sts_monologg-koelectra-base-v3-discriminator-2021-10-28_04-48-52


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-10-28 05:20:38 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 3:
2021-10-28 05:20:44 - Cosine-Similarity :	Pearson: 0.6971	Spearman: 0.6975
2021-10-28 05:20:44 - Manhattan-Distance:	Pearson: 0.6792	Spearman: 0.6660
2021-10-28 05:20:44 - Euclidean-Distance:	Pearson: 0.6805	Spearman: 0.6678
2021-10-28 05:20:44 - Dot-Product-Similarity:	Pearson: 0.6340	Spearman: 0.6194
2021-10-28 05:20:44 - Save model to output/training_klue_sts_monologg-koelectra-base-v3-discriminator-2021-10-28_04-48-52


# 형태소 분석 및 품사 필터링 (명사, 동사, 형용사)

In [16]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 448 kB 44.6 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-10-28 05:20:53--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00

In [8]:
from konlpy.tag import *
okt = Okt()

In [19]:
docs = [
    "1992년 7월 8일 손흥민은 강원도 춘천시 후평동에서 아버지 손웅정과 어머니 길은자의 차남으로 태어나 그곳에서 자랐다.",
    "형은 손흥윤이다.",
    "춘천 부안초등학교를 졸업했고, 춘천 후평중학교에 입학한 후 2학년때 원주 육민관중학교 축구부에 들어가기 위해 전학하여 졸업하였으며, 2008년 당시 FC 서울의 U-18팀이었던 동북고등학교 축구부에서 선수 활동 중 대한축구협회 우수선수 해외유학 프로젝트에 선발되어 2008년 8월 독일 분데스리가의 함부르크 유소년팀에 입단하였다.",
    "함부르크 유스팀 주전 공격수로 2008년 6월 네덜란드에서 열린 4개국 경기에서 4게임에 출전, 3골을 터뜨렸다.",
    "1년간의 유학 후 2009년 8월 한국으로 돌아온 후 10월에 개막한 FIFA U-17 월드컵에 출전하여 3골을 터트리며 한국을 8강으로 이끌었다.",
    "그해 11월 함부르크의 정식 유소년팀 선수 계약을 체결하였으며 독일 U-19 리그 4경기 2골을 넣고 2군 리그에 출전을 시작했다.",
    "독일 U-19 리그에서 손흥민은 11경기 6골, 2부 리그에서는 6경기 1골을 넣으며 재능을 인정받아 2010년 6월 17세의 나이로 함부르크의 1군 팀 훈련에 참가, 프리시즌 활약으로 함부르크와 정식 계약을 한 후 10월 18세에 함부르크 1군 소속으로 독일 분데스리가에 데뷔하였다.",
]
document_embeddings = model.encode(docs)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [32]:
result_arr = []
result = []
for sentence in docs:
  clean_words = []
  for word in okt.pos(sentence, stem=True):
    if word[1] in ['Noun', 'Verb', 'Adjective']: #명사, 동사, 형용사
      clean_words.append(word[0])
  print(clean_words)
  temp_sentence = ' '.join(clean_words)
  result_arr.append(clean_words)
  result.append(temp_sentence)

['손흥민', '강원도', '춘천시', '후평동', '아버지', '손웅정', '어머니', '길다', '자의', '차남', '태어나다', '곳', '자르다']
['형', '손흥윤']
['춘천', '부안', '초등학교', '졸업', '하다', '춘천', '후평', '중학교', '입학', '후', '학년', '때', '원주', '민', '관중', '학교', '축구', '부', '들어가다', '위해', '전학', '하다', '졸업', '하다', '당시', '서울', '팀', '이다', '동북', '고등학교', '축구', '부', '선수', '활동', '중', '축구', '협회', '우수', '선수', '해외', '유학', '프로젝트', '선발', '되어다', '독일', '분데스리가', '함부르크', '유', '소년', '팀', '입단', '하다']
['함부르크', '유', '스팀', '주전', '공격수', '네덜란드', '열리다', '개국', '경기', '게임', '출전', '골', '터뜨리다']
['유학', '후', '한국', '돌아오다', '후', '개막', '월드컵', '출전', '하다', '골', '터', '트리', '한국', '강', '이끌다']
['함부르크', '정식', '유', '소년', '팀', '선수', '계약', '체결', '하다', '독일', '리그', '경기', '골', '넣다', '군', '리그', '출전', '시작', '하다']
['독일', '리그', '손흥민', '경기', '골', '부', '리그', '경기', '골', '넣다', '재능', '인정받다', '세', '나이', '함부르크', '군', '팀', '훈련', '참가', '프리', '시즌', '활약', '함부르크', '정식', '계약', '하다', '후', '세', '함부르크', '군', '소속', '독일', '분데스리가', '데뷔', '하다']


# 코사인 유사도 기반으로 한 국소적 응집도 측정 _ 블로그


In [34]:
import pandas as pd

blog_data = pd.read_csv('/content/drive/MyDrive/rsn_nlp_project/lucy_blog.csv', encoding='utf-8')

In [35]:
contents = blog_data

In [36]:
data_colname = 'contents'

contents[data_colname] = contents[data_colname].str.replace('\"', "",regex=True)
contents[data_colname] = contents[data_colname].str.replace("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+)", " ", regex=True) #이메일 주소에서 오는 유사도 차단
contents[data_colname] = contents[data_colname].str.replace("\n"," ")
contents[data_colname] = contents[data_colname].str.replace("\r"," ")
contents[data_colname] = contents[data_colname].str.replace("\t"," ")
contents[data_colname] = contents[data_colname].str.replace("\u200b","")

In [37]:
!pip install kss==2.5.1

In [38]:
contents['morphs'] = -1
contents['morphs_arr'] = -1

In [ ]:
from kss import split_sentences

for idx, doc in enumerate(contents['contents'][10:50]):
  print(idx+10)
  sentences = split_sentences(doc)
  result_arr = []
  result = []
  for sentence in sentences:
    clean_words = []
    for word in okt.pos(sentence, stem=True):
      if word[1] in ['Noun', 'Verb', 'Adjective']: #명사, 동사, 형용사
        clean_words.append(word[0])
    temp_sentence = ' '.join(clean_words)
    result_arr.append(clean_words)
    result.append(temp_sentence)
  contents['morphs'][idx+10] = result
  contents['morphs_arr'][idx+10] = result_arr

In [ ]:
contents[10:50].head()

In [43]:
import numpy
doc_all_coherence=[]
for index, doc in enumerate(contents['morphs'][10:50]):
  all_coherence = []
  for i, sentence in enumerate(doc):
    print(i)
    if(i==len(doc)-1): 
      break
    query_embedding_a = model.encode(doc[i])
    query_embedding_b = model.encode(doc[i+1])

    # 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
    cos_scores = util.pytorch_cos_sim(query_embedding_a, query_embedding_b)[0]
    all_coherence.append(cos_scores)
  print('문서 평균 코사인 유사도: ',numpy.mean(all_coherence))
  print('표준 편차: ',numpy.std(all_coherence))
  doc_all_coherence.append(numpy.mean(all_coherence))

print(doc_all_coherence)
print('전체 평균 코사인 유사도: ',numpy.mean(doc_all_coherence))
print('표준 편차: ',numpy.std(doc_all_coherence))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([0.9589])
9


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([0.9682])
10


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([0.9835])
21


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([0.4453])
22
문서 평균 코사인 유사도:  0.9411015
표준 편차:  0.108678214
[0.9225891, 0.9478051, 0.9295293, 0.9401403, 0.9199804, 0.96302205, 0.97050905, 0.964243, 0.9609573, 0.9645771, 0.9699687, 0.942205, 0.9638036, 0.7887189, 0.9193934, 0.94676185, 0.9441635, 0.9283516, 0.95342106, 0.947589, 0.9610463, 0.9184532, 0.9563324, 0.9455862, 0.95273846, 0.9604367, 0.92935514, 0.86144435, 0.9578268, 0.9739621, nan, nan, 0.9346834, 0.94054914, 0.9703694, 0.96593857, 0.9395022, 0.94725513, 0.93855864, 0.9411015]
전체 평균 코사인 유사도:  nan
표준 편차:  nan


In [44]:
list = [0.9225891, 0.9478051, 0.9295293, 0.9401403, 0.9199804, 0.96302205, 0.97050905, 0.964243, 0.9609573, 0.9645771, 0.9699687, 0.942205, 0.9638036, 0.7887189, 0.9193934, 0.94676185, 0.9441635, 0.9283516, 0.95342106, 0.947589, 0.9610463, 0.9184532, 0.9563324, 0.9455862, 0.95273846, 0.9604367, 0.92935514, 0.86144435, 0.9578268, 0.9739621, 0.9346834, 0.94054914, 0.9703694, 0.96593857, 0.9395022, 0.94725513, 0.93855864, 0.9411015]

In [45]:
print('전체 평균 코사인 유사도: ',numpy.mean(list))
print('표준 편차: ',numpy.std(list))

전체 평균 코사인 유사도:  0.9416544457894737
표준 편차:  0.03256547828364382


# 코사인 유사도 기반으로 한 국소적 응집도 측정 _ 언론


In [46]:
import pandas as pd

news_data = pd.read_csv('/content/drive/MyDrive/rsn_nlp_project/sample_0824.csv', encoding='utf-8')

In [47]:
contents = news_data['contents'][30000:30050]

In [48]:
contents = contents.reset_index()
contents = contents.drop('index', axis=1)

In [49]:
#이어서 contents 전처리(정제)
data_colname = 'contents'
contents[data_colname] = contents[data_colname].str.replace("\(.*\)|\s-\s.*"," " ,regex=True)
contents[data_colname] = contents[data_colname].str.replace("\[.*\]|\s-\s.*"," ",regex=True)
contents[data_colname] = contents[data_colname].str.replace("\<.*\>|\s-\s.*"," ",regex=True)
contents[data_colname] = contents[data_colname].str.replace("무단전재 및 재배포 금지"," ",regex=True)
contents[data_colname] = contents[data_colname].str.replace("무단 전재 및 재배포 금지"," ",regex=True)
contents[data_colname] = contents[data_colname].str.replace("©"," ",regex=True)
contents[data_colname] = contents[data_colname].str.replace("ⓒ"," ",regex=True)
contents[data_colname] = contents[data_colname].str.replace("저작권자"," ",regex=True)
contents[data_colname] = contents[data_colname].str.replace(".* 기자", " ", regex=True) #기자 이름에서 오는 유사도 차단
contents[data_colname] = contents[data_colname].str.replace("사진 = .*", " ", regex=True) #사진 첨부 문구 삭제
contents[data_colname] = contents[data_colname].str.replace("사진=.*", " ", regex=True) #사진 첨부 문구 삭제
contents[data_colname] = contents[data_colname].str.replace('\"', "",regex=True)
contents[data_colname] = contents[data_colname].str.replace("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+)", " ", regex=True) #이메일 주소에서 오는 유사도 차단
contents[data_colname] = contents[data_colname].str.replace("\n"," ")
contents[data_colname] = contents[data_colname].str.replace("\r"," ")
contents[data_colname] = contents[data_colname].str.replace("\t"," ")
contents[data_colname] = contents[data_colname].str.replace( "\’" , "", regex=True)
contents[data_colname] = contents[data_colname].str.replace("[ ]{2,}"," ",regex=True)

In [50]:
contents['morphs'] = -1
contents['morphs_arr'] = -1

In [51]:
contents.head()

,contents,morphs,morphs_arr
0,한상혁 방통위원장 후보자 허위조작정보 개선책 고민 방송통신위원회 위원장 후보자로 ...,-1,-1
1,"네오위즈, 2분기 영업익 82억원…전년비 4%↓ 네오위즈는 2분기 연결기준 영업이...",-1,-1
2,"한샘, 한샘몰 앱서 출석체크·타임딜 등 이벤트 열어 한샘은 오는 31일까지 자사 ...",-1,-1
3,"BC카드, '페이북 QR결제' 대학생 아이디어 공모전 시상 BC카드 부문으로 참가...",-1,-1
4,"케이블TV협회, 2분기 지역채널 우수프로그램 9편 선정 한국케이블TV방송협회는 9...",-1,-1


In [52]:
from kss import split_sentences

for idx, doc in enumerate(contents['contents']):
  sentences = split_sentences(doc)
  result_arr = []
  result = []
  for sentence in sentences:
    clean_words = []
    for word in okt.pos(sentence, stem=True):
      if word[1] in ['Noun', 'Verb', 'Adjective']: #명사, 동사, 형용사
        clean_words.append(word[0])
    temp_sentence = ' '.join(clean_words)
    result_arr.append(clean_words)
    result.append(temp_sentence)
  contents['morphs'][idx] = result
  contents['morphs_arr'][idx] = result_arr

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instan

In [72]:
import math

import numpy
doc_all_coherence=[]
for index, doc in enumerate(contents['morphs'][:10]):
  all_coherence = []
  if(len(doc)==1):
    continue
  for i, sentence in enumerate(doc):
    if(i==len(doc)-1): 
      break
    query_embedding_a = model.encode(doc[i])
    query_embedding_b = model.encode(doc[i+1])

    # 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
    cos_scores = util.pytorch_cos_sim(query_embedding_a, query_embedding_b)[0]
    all_coherence.append(cos_scores)
  print(index,' 번째 문서 평균 코사인 유사도: ',numpy.mean(all_coherence))
  print('표준 편차: ',numpy.std(all_coherence))
  doc_all_coherence.append(numpy.mean(all_coherence))

print(doc_all_coherence)
print('전체 평균 코사인 유사도: ',numpy.mean(doc_all_coherence))
print('표준 편차: ',numpy.std(doc_all_coherence))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

9  번째 문서 평균 코사인 유사도:  0.95879215
표준 편차:  0.014230158
[0.97018397, 0.9192805, 0.9481003, 0.9710456, 0.97461224, 0.96147186, 0.9711718, 0.973331, 0.95879215]
전체 평균 코사인 유사도:  0.9608877
표준 편차:  0.016792757
